## **Práctica 2: Crear y entrenar CNNs desde cero**

<hr>

Las redes de neuronas convolucionales o simplemente **redes convolucionales** (CNNs, del inglés *convolutional neural networks*), son un tipo de redes neuronales profundas. De hecho son muy similares a estas, dado que también tienen una capa de entrada, una o varias capas ocultas, y una capa de salida,  definidas todas ellas por unos parámetros que se aprenden durante la fase de entrenamiento. Por tanto, una CNN se puede expresar como una función derivable que utiliza los píxeles de una imagen de entrada para obtener unas probabilidades para cada una de las clases objetivo (problema de clasificación) o un valor numérico (problema de regresión).

A continuación, vamos a ver un ejemplo en el que se crea y entrena una CNN desde cero, utilizando la librería [TensorFlow](https://www.tensorflow.org/). También veremos cómo utilizar la herramienta [TensorBoard](https://www.tensorflow.org/tensorboard) para visualizar métricas y otra información de interés durante el entrenamiento.

################# APUNTES"""""""""
imagen -> base cnvolucional ->mapa de caracteristias-> vector de caracteristicas

lo qu teemos que poasar  ala parte completamente concectada es un mapa de caracteriscitas, con flatten(aplane extraiga informacion) o global pooling

Antes de empezar, vamos a utilizar el método [set_random_seed()](https://www.tensorflow.org/api_docs/python/tf/keras/utils/set_random_seed) para establecer el valor de la **semilla** y garantizar la reproducibilidad de los resultados.

In [46]:
import tensorflow as tf
# To-Do: Fijar la semilla
seed=42
tf.keras.utils.set_random_seed(seed)

<hr>

### **1. Conjunto de datos**

En esta práctica vamos a utilizar un conjunto de datos para clasificación de imágenes denominado [CIFAR10](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/cifar10), disponible para descarga en `TensorFlow`. Este conjunto está compuesto por 50.000 imágenes de entrenamiento y 10.000 imágenes de test. Se trata de imágenes en color, de dimensiones espaciales 32x32 y etiquetadas en 10 categorías.

En la web de `TensorFlow` puedes encontrar otros [conjuntos de datos](https://www.tensorflow.org/api_docs/python/tf/keras/datasets).

In [47]:
# To-Do: Cargar el conjunto de datos CIFAR10 (entrenamiento y test)
dataset= tf.keras.datasets.cifar10
(x_train, y_train), (x_test, y_test)= dataset.load_data()
# Verificar las dimensiones
# Verificar dimensiones
print(f"Conjunto de entrenamiento: {x_train.shape}, {y_train.shape}")
print(f"Conjunto de prueba: {x_test.shape}, {y_test.shape}")

Conjunto de entrenamiento: (50000, 32, 32, 3), (50000, 1)
Conjunto de prueba: (10000, 32, 32, 3), (10000, 1)


El siguiente paso consiste en codificar las diferentes clases utilizando el método [`to_categorical()`](https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical), disponible en `TensorFlow`.

In [48]:
# To-Do: Codificar las clases (one-hot enconding)
from tensorflow.keras.utils import to_categorical
y_train_one_hot=to_categorical(y_train, num_classes=10)
y_test_one_hot=to_categorical(y_test, num_classes=10)

A continuación, vamos a dividir el conjunto de entrenamiento para crear la partición de validación (proporción 80:20). Para ello utilizaremos el método [train_test_split()](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html), disponible en la librería `scikit-learn`.

In [49]:
from sklearn.model_selection import train_test_split
# To-Do: Dividir el conjunto de entrenamiento en entrenamiento y validación
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train_one_hot, train_size=0.8,random_state=seed,stratify=y_train)
# Mostrar las dimensiones de cada conjunto
print(f"Nuevo conjunto de entrenamiento: {x_train.shape}, {y_train.shape}")
print(f"Conjunto de validación: {x_val.shape}, {y_val.shape}")
print(f"Conjunto de prueba: {x_test.shape}, {y_test_one_hot.shape}")

Nuevo conjunto de entrenamiento: (40000, 32, 32, 3), (40000, 10)
Conjunto de validación: (10000, 32, 32, 3), (10000, 10)
Conjunto de prueba: (10000, 32, 32, 3), (10000, 10)


Por último, vamos a normalizar los datos de entrada y generar los *batches* necesarios para entrenar la red que se define a continuación. Para ello utilizaremos, en primer lugar, el método [`from_tensor_slices()`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_tensor_slices) de la clase [`Dataset`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset), que permite crear un conjunto de datos a partir de datos de entrada que ya están en memoria.

In [67]:
# To-Do: Crear los conjuntos de datos (Dataset)
train_data = tf.data.Dataset.from_tensor_slices((x_train,y_train))
val_data=tf.data.Dataset.from_tensor_slices((x_val,y_val))
test_data=tf.data.Dataset.from_tensor_slices((x_test,y_test_one_hot))

A continuación, vamos a definir una función `normalize_images(images, labels)` que nos permita normalizar las imágenes (media, desviación típica).

In [69]:
mean=x_train.mean()
std=x_train.std()
# To-Do: Normalizar las imágenes (media y desviación típica)
def normalize_images(images,labels):
    images = (images - mean) / std      
    return images, labels


Una vez definida la función, utilizaremos el método [`map()`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#map) de la clase `Dataset` para normalizar los tres conjuntos de datos (entrenamiento, validación, test).

In [70]:
# To-Do: Normalizar los datos (normalización global, no por canales)
train_data=train_data.map(normalize_images)
val_data=val_data.map(normalize_images)
test_data=test_data.map(normalize_images)

Por último, utilizaremos el método [`batch()`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#batch) de la clase `Dataset` para crear los batches de las tres particiones de datos, con un tamaño de 256.

In [71]:
# To-Do: Preparar los lotes (batches) para las tres particiones de datos
batch_size=256
train_data_batches=train_data.batch(batch_size)
val_data_batches=val_data.batch(batch_size)
test_data_batches=test_data.batch(batch_size)

<hr>

### **2. Red convolucional**

El siguiente paso consiste en crear una sencilla CNN utilizando las siguientes capas:

*   [Capa de entrada](https://www.tensorflow.org/api_docs/python/tf/keras/layers/InputLayer): `InputLayer(shape=None)` crea una capa de entrada con el tamaño indicado en `shape`, que será una tupla de números enteros.

*   [Capa convolucional](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D): `Conv2D(n_filters, kernel_size)` crea una capa con `n_filters` de tamaño `kernel_size` que se aplican a los datos de entrada para producir un tensor de salidas. Si `use_bias` es `True`, se crea un vector de sesgo y se suma a las salidas. Si `activation` no es `None`, también se aplica la función de activación especificada a las salidas. Otros parámetros relevantes:
> * `strides`: un entero o tupla/lista de dos enteros que especifique el paso de la convolución a lo largo del alto y ancho del volumen de entrada. Especificar un entero implica que se usará el mismo valor para todas las dimensiones espaciales (alto, ancho).
> * `padding`: `valid`, que significa sin relleno; o `same`, que da como resultado un relleno de ceros uniforme (izquieda/derecha y arriba/abajo). Si `padding='same'`y `strides=1`, la salida tiene el mismo tamaño que la entrada.
> * `activation`: función de activación (`relu`, `sigmoid`, etc.) Por defecto, `activation=None` (es decir, no se utiliza función de activación).
> * `input_shape`: cuando se utiliza como primera capa del modelo, es necesario indicar las dimensiones del volumen de entrada; por ejemplo, `input_shape=(128, 128, 3)` para imágenes RGB de 128x128 en formato `data_format="channels_last"`.

*   [Capa max-pooling](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D): `MaxPool2D()` reduce los datos de entrada a lo largo de las dimensiones espaciales (alto, ancho) utilizando, para cada canal de la entrada, el valor máximo sobre una ventana de tamaño `pool_size` (por defecto, `pool_size=2`). La ventana se desplaza a lo largo de cada dimensión utilizando el valor del parámetro `strides` (por defecto, `strides=pool_size`).

*   [Capa flatten](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten): `Flatten()` aplana la entrada, convirtiendo un volumen en vector.

*   [Capa completamente conectada](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense): `Dense(units)` crea una capa complementamente conectada con el número de neuronas especificado en `units`. Otros parámetros relevantes:
> * `activation`: función de activación (`relu`, `sigmoid`, `softmax`, etc.)
> * `use_bias`: Booleano que indica si la capa utiliza un vector de sesgo (`True`, valor por defecto) o no (`False`).

A continuación, vamos a definir una función `get_model()` que permita crear un [modelo secuencial](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential) con la siguiente arquitectura:
* La capa de entrada.
* Dos capas convolucionales de 32 filtros (dimensiones 3x3), con ReLU como función de activación.
* Una capa max-pooling con tamaño de ventana 2.
* Dos capas convolucionales de 64 filtros (dimensiones 3x3), con ReLU como función de activación.
* Una capa max-pooling con tamaño de ventana 2.
* Una capa que aplane el volumen para producir un vector.
* Una capa completamente conectada 512 neuronas, con ReLU como función de activación.
* La capa salida.

In [54]:
# To-Do: Crear un modelo secuencial, compuesto por una secuencia lineal de capas

Por último, vamos a crear el modelo invocando la función anterior y a utilizar el método [`summary()`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#summary) para imprimir una representación en modo texto de la arquitectura definida. Con este método es posible visualizar también el número de parámetros de cada capa de la red.

In [55]:
# To-Do: Crear el modelo e imprimir su representación en modo texto

<hr>

### **3. Entrenamiento**

Una vez definida la arquitectura de la CNN, el siguiente paso es configurar el modelo para el entrenamiento. Para ello utilizaremos el método [`compile()`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#compile), siendo estos algunos de sus parámetros más relevantes:

* `optimizer`: nombre del optimizador (`Adam`, `RMSProp`, etc.) y tasa de aprendizaje (`learning_rate`). En la web de `TensorFlow` puedes encontrar otros [optimizadores](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers).
* `loss`: función de pérdida (`mean_squared_error`, `binary_crossentropy`, `categorical_crossentropy`, etc.). En la web de `TensorFlow` puedes encontrar otras [funciones de pérdida](https://www.tensorflow.org/api_docs/python/tf/keras/losses).
* `metrics`: métricas que se evalúan para los datos de entrenamiento y validación (`accuracy`, etc.). En la web de `TensorFlow` puedes encontrar otras [métricas](https://www.tensorflow.org/api_docs/python/tf/keras/metrics).

Para configurar el proceso de aprendizaje, utiliza el optimizador [Adam](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam), una tasa de aprendizaje de `0.001` y una función de pérdida apropiada, teniendo en cuenta la tarea a resolver.


In [56]:
# To-Do: Configurar el proceso de aprendizaje

A continuación, vamos a entrenar el modelo para buscar los parámetros que hagan mínima la función de pérdida. Para ello utilizaremos el método [`fit()`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit), que necesita que le suministremos los datos de entrenamiento y validación, y el número de *epochs*.

Al finalizar cada *epoch* se mostrará una línea con, por ejemplo, la siguiente información:

> `11s - loss: 1.5593 - acc: 0.4327 - val_loss: 1.2593 - val_acc: 0.5539`

El primer número (`11s`) son los segundos que le ha llevado completar la epoch. `loss` es el valor de la función de pérdida calculado sobre el conjunto de entrenamiento y `val_loss` lo mismo pero calculado sobre el conjunto de validación (cuanto menor, mejor). `acc` y `val_acc` son el ratio de acierto (*accuracy*) calculado sobre el conjunto de entrenamiento y validación, respectivamente (cuanto mayor, mejor).

In [57]:
# To-Do: Entrenar el modelo

Por último, vamos a imprimir el error mínimo de entrenamiento y validación.

In [58]:
# To-Do: Imprimir el error mínimo (entrenamiento y validación)

<hr>

### **4. Evaluación**

Hemos visto cómo crear y entrenar una CNN desde cero, utilizando una configuración de hiperparámetros que no necesariamente es la mejor. Lo ideal sería realizar una búsqueda de hiperparámetros y, una vez obtenida la mejor configuración, evaluar el modelo sobre el conjunto de test y así obtener el resultado final.

A continuación, utilizaremos el método [`evaluate()`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#evaluate) para evaluar el modelo final en el conjunto de test.

In [59]:
# To-Do: Evaluar el modelo en el conjunto de test

Por último, además de analizar el error obtenido, podemos hacer predicciones con el modelo entrenado. Para ello utilizaremos el método [`predict()`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#predict), al que le suministraremos los datos sobre los que realizar las predicciones (en este caso, los datos de test).

In [60]:
# To-Do: Obtener las predicciones para todos los datos de test y mostrar los valores obtenidos para las N primeras imágenes

<hr>

### **5. TensorBoard**

TensorBoard es una herramienta de visualización para analizar, depurar y optimizar modelos en TensorFlow. Facilita la interpretación de datos al mostrar:

* Métricas de entrenamiento.
* Histogramas de pesos, sesgos y otros tensores.

Para utilizar esta herramienta, tendremos que definir el [callback de TensorBoard](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard) y asignarlo durante el entrenamiento. En la web de `TensorFlow` puedes encontrar otros [callbacks](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks).

**NOTA:** Con el objetivo de almacenar los ficheros de log de varios procesos de entrenamiento, utiliza el siguiente código para generar un log con la información de fecha y hora actual.

```python
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")  # directorio de logs
```

In [61]:
# To-Do: Entrenar el modelo utilizando el callback de TensorBoard

Por último, podrás visualizar los resultados iniciando TensorBoard desde la línea de comandos.

In [62]:
%load_ext tensorboard
%tensorboard --logdir logs/fit

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 4124), started 0:00:31 ago. (Use '!kill 4124' to kill it.)

<hr>

### **6. Ejercicios**

A continuación, se proponen una serie de ejercicios para su resolución.

**EJERCICIO 1**

Modifica las capas convolucionales del modelo original utilizando el parámetro `padding='same'`.

¿Cómo afecta este cambio en la arquitectura? Analiza las dimensiones de salida y el número de parámetros de cada capa.

In [63]:
# To-Do: Solución al ejercicio 1

**EJERCICIO 2**

Partiendo del modelo obtenido en el ejercicio 1, añade un nuevo bloque de capas convolucionales antes de la capa *flatten* con las siguientes indicaciones: dos capas convolucionales de 128 filtros cada una y dimensiones espaciales 3x3, sin utilizar *padding*. Estas dos capas deberán ir seguidas de una capa *max-pooling* con tamaño de ventana 2.

¿Qué impacto tiene en el número de parámetros total añadir estas tres capas? ¿Sería posible hacer esta modificación en el modelo original?

In [64]:
# To-Do: Solución al ejercicio 2

**EJERCICIO 3**

Partiendo del modelo original, realiza diferentes pruebas para intentar mejorar los resultados. Algunas sugerencias:

*  Realiza una búsqueda exhaustiva de hiperparámetros (tasa de aprendizaje, optimizador, etc.).

*  Modifica la secuencia de capas (capas convolucionales, número de filtros, capas completamente conectadas, etc.).

*  Utiliza técnicas que ayuden a prevenir el sobreajuste como, por ejemplo, capas `Dropout` después de las capas *pooling* y/o entre las capas completamente conectadas.
> * [Capa Dropout](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout): `Dropout(rate)` pone a 0, aleatoriamente, las entradas con una frecuencia de `rate` (número real entre 0 y 1) en cada paso del proceso de entrenamiento, reduciendo así el sobreajuste.

*  Utiliza técnicas que ayuden a acelerar el entrenamiento como, por ejemplo, capas `BatchNormalization` antes de las capas de activación `ReLU`.
> * [Capa BatchNormalization](https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization): `BatchNormalization()` normaliza las entradas.

In [65]:
# To-Do: Solución al ejercicio 3